In [3]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Annotated
from langchain_core.messages import BaseMessage,HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

from langchain_community.chat_models import ChatOllama

from langgraph.checkpoint.memory import MemorySaver

In [4]:
#baseMessage contains all Human,Ai,System and tool message
from langgraph.graph import add_messages
class chat_state(TypedDict):
    messages:Annotated[list[BaseMessage],add_messages]

In [5]:
model=ChatOllama(model='llama2')

def chat_node(state:chat_state):
    messages=state['messages']
    result=model.invoke(messages)
    return {'messages':[result]}

C:\Users\sinanant\AppData\Local\Temp\ipykernel_19176\2160552625.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model=ChatOllama(model='llama2')


In [6]:
checkpointer=MemorySaver()
graph=StateGraph(chat_state)

graph.add_node('chat_node',chat_node)

graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

chatbot=graph.compile(checkpointer=checkpointer)

In [55]:
result['messages'][-1].content


'\nThe capital of India is New Delhi.'

In [59]:
thread_id='1'

while True:
    user_msg=input('Type here: ')
    print('User:',user_msg)
    if user_msg.strip().lower() in ['exit','quit','bye']:
        break
    config={'configurable':{'thread_id':thread_id}}
    result=chatbot.invoke({'messages':[HumanMessage(content=user_msg)]},config=config)
    
    print('AI:',result['messages'][-1].content)

User: HI,MY NAME IS ANANT SINGH
AI: 
Nice to meet you, Anant Singh! *smiles* How are you today?
User: WHAT IS MY NAME?
AI: 
Your name is Anant Singh.
User: 2+3=?
AI: 
2 + 3 = 5.
User: WHAT IS 3 + PREVIOUS RESULT
AI: 
The previous result was 5, so 3 + 5 = 8.
User: exit


In [60]:
chatbot.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='HI,MY NAME IS ANANT SINGH', additional_kwargs={}, response_metadata={}, id='989ec1a0-e661-4cb1-8971-8fd7006b3a83'), AIMessage(content='\nNice to meet you, Anant Singh! *smiles* How are you today?', additional_kwargs={}, response_metadata={'model': 'llama2', 'created_at': '2025-07-25T17:08:41.5014106Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7442514000, 'load_duration': 2855228200, 'prompt_eval_count': 30, 'prompt_eval_duration': 2282612300, 'eval_count': 21, 'eval_duration': 2303276000, 'model_name': 'llama2'}, id='run--0c8cfa0e-982e-49a7-ab85-69a34362fa37-0', usage_metadata={'input_tokens': 30, 'output_tokens': 21, 'total_tokens': 51}), HumanMessage(content='WHAT IS MY NAME?', additional_kwargs={}, response_metadata={}, id='6de4c861-ba21-4fa7-830b-2bcf40ed1fbf'), AIMessage(content='\nYour name is Anant Singh.', additional_kwargs={}, response_metadata={'model': 'llama2', 'created_at': '2025-07-25T17:08:51.4050215Z